In [7]:
import mlflow
from mlflow.tracking import MlflowClient

EXPERIMENT_NAME =' [FR] [Paris] [P2M] EXP '

# Indicate mlflow to log to remote server
mlflow.set_tracking_uri("https://mlflow.lewagon.co/")

client = MlflowClient()

try:
    experiment_id = client.create_experiment(EXPERIMENT_NAME)
except BaseException:
    experiment_id = client.get_experiment_by_name(
        EXPERIMENT_NAME).experiment_id

yourname = 'PIERRE2M'

if yourname is None:
    print("please define your name, il will be used as a parameter to log")

for model in ["linear", "Randomforest"]:
    
    
    
    run = client.create_run(experiment_id)
    client.log_metric(run.info.run_id, "rmse", 4.5)
    client.log_param(run.info.run_id, "model", model)
    client.log_param(run.info.run_id, "student_name", yourname)
    client.set_experiment_tag('test',45)
    client.log_param(run.info.run_id,"alpha", 0)
    client.log_param(run.info.run_id,"l1_ratio", 1)
    client.log_metric(run.info.run_id,"rmse", 2)
    client.log_metric(run.info.run_id,"r2", 3)
    client.log_metric(run.info.run_id,"mae", 4)




TypeError: set_experiment_tag() missing 1 required positional argument: 'value'